In [1]:
import tensorflow as tf
from python_speech_features import *
import wave
import pyaudio
import numpy as np

In [2]:
model_epd = tf.keras.models.load_model(r"D:\语音识别\念佛计数\h5\EPD_MODEL.h5")
model_recognition = tf.keras.models.load_model(r"D:\语音识别\念佛计数\h5\ACOUSTIC_32_MODEL_观音菩萨.h5")
model_countor = tf.keras.models.load_model(r"D:\语音识别\念佛计数\h5\LANG_MODEL_观音菩萨.h5")

In [3]:
class IQueue:
    def __init__(self, maxSize):
        self.maxSize = maxSize
        self.arr = [0] * maxSize
        self.pointer = 0
        self.size = 0
        
    def getArray(self):       
        a = []
        a.extend(self.arr[self.pointer:])
        a.extend(self.arr[:self.pointer])        
        return a
    
    def push(self, value):
        if self.pointer == self.maxSize:           
            self.pointer = 0            
        self.arr[self.pointer] = value
        self.pointer += 1
        if self.size < self.maxSize:
            self.size += 1
            
def preprocessingForEpd(soundBytes):
    mean = soundBytes.mean()
    std = soundBytes.std()
    soundBytes = (soundBytes - mean)/std  
    return soundBytes

def epd_predict(data, seq_length):
    soundBytes = np.fromstring(np.array(data), dtype=np.int16)    
    pfe = preprocessingForEpd(soundBytes).reshape(1,seq_length,1)    
    epd = int(np.round(model_epd.predict(pfe)))   
    return epd, soundBytes

def preprocessingForRecognition(soundBytes):
    soundBytes = (soundBytes - soundBytes.mean())/soundBytes.std()
    mfcc_feat0 =  mfcc(soundBytes)
    mfcc_feat1 = delta(mfcc_feat0, 1)
    mfcc_feat2 = delta(mfcc_feat0, 2)    
    feature = np.hstack((mfcc_feat0, mfcc_feat1, mfcc_feat2))    
    feature = feature.reshape(1,-1,39,1)
    return feature

def recognition_predict(soundBytes):    
    pfr = preprocessingForRecognition(soundBytes)    
    rp = model_recognition.predict(pfr)   
    return rp

def countor_predict(q):
    slices_rps = q.getArray()      
    rp_reshape = np.reshape((slices_rps), (1,1,160))    
    ctor = int(np.round(model_countor.predict(rp_reshape)))
    return ctor

In [4]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
RECORD_SECONDS = 1000
pad = pyaudio.PyAudio()
stream = pad.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

In [ ]:
print("开始录音")
q = IQueue(5)
last_ctor = 0
countor = 0
for t in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)    
    epd, soundBytes = epd_predict(data, CHUNK)     
    #print(epd, end="")
    if epd == 1:
        rp = recognition_predict(soundBytes)       
        q.push(rp)
        if q.size == 5 :
            ctor = countor_predict(q)      
            print(ctor, end="")
            
        
print("结束录音")

开始录音


c:\program files\python37\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


0000000000000000000000000000000000000000000000111111110000000000000000000000000000000000000111000000000000000000000011111100011111111000000000000000011111010000000111100000000001110000000000000111111111000000000000000000000000000000001110000000000000000000000000000010000000000000000111111000